In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from flask import Flask, render_template, request, redirect, url_for, jsonify
import pickle

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
data=pd.read_csv(r"C:\Users\Srinivasan\Desktop\SRP - SRINEDHI\data\newdata.csv")

In [3]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn import metrics

In [4]:
data.head()

,STATE/UT,DISTRICT,YEAR,MURDER,ATTEMPT TO MURDER,CULPABLE HOMICIDE NOT AMOUNTING TO MURDER,RAPE,CUSTODIAL RAPE,OTHER RAPE,KIDNAPPING & ABDUCTION,...,ARSON,HURT/GREVIOUS HURT,DOWRY DEATHS,ASSAULT ON WOMEN WITH INTENT TO OUTRAGE HER MODESTY,INSULT TO MODESTY OF WOMEN,CRUELTY BY HUSBAND OR HIS RELATIVES,IMPORTATION OF GIRLS FROM FOREIGN COUNTRIES,CAUSING DEATH BY NEGLIGENCE,OTHER IPC CRIMES,TOTAL IPC CRIMES
0,ANDHRA PRADESH,ADILABAD,2012,81,80,14,88,0,88,82,...,35,2172,7,90,155,498,0,400,1079,5710
1,ANDHRA PRADESH,ANANTAPUR,2012,173,123,3,18,0,18,74,...,24,2018,19,195,8,130,0,576,1043,5441
2,ANDHRA PRADESH,CHITTOOR,2012,91,75,5,28,0,28,56,...,27,622,33,90,269,339,0,520,1398,4465
3,ANDHRA PRADESH,CUDDAPAH,2012,118,96,2,26,0,26,76,...,6,2355,13,217,37,120,0,500,1360,6110
4,ANDHRA PRADESH,CYBERABAD,2012,198,113,10,75,0,75,158,...,48,4013,45,183,282,976,0,1095,3155,17979


In [5]:
data.keys()

Index(['STATE/UT', 'DISTRICT', 'YEAR', 'MURDER', 'ATTEMPT TO MURDER',
       'CULPABLE HOMICIDE NOT AMOUNTING TO MURDER', 'RAPE', 'CUSTODIAL RAPE',
       'OTHER RAPE', 'KIDNAPPING & ABDUCTION',
       'KIDNAPPING AND ABDUCTION OF WOMEN AND GIRLS',
       'KIDNAPPING AND ABDUCTION OF OTHERS', 'DACOITY',
       'PREPARATION AND ASSEMBLY FOR DACOITY', 'ROBBERY', 'BURGLARY', 'THEFT',
       'AUTO THEFT', 'OTHER THEFT', 'RIOTS', 'CRIMINAL BREACH OF TRUST',
       'CHEATING', 'COUNTERFIETING', 'ARSON', 'HURT/GREVIOUS HURT',
       'DOWRY DEATHS', 'ASSAULT ON WOMEN WITH INTENT TO OUTRAGE HER MODESTY',
       'INSULT TO MODESTY OF WOMEN', 'CRUELTY BY HUSBAND OR HIS RELATIVES',
       'IMPORTATION OF GIRLS FROM FOREIGN COUNTRIES',
       'CAUSING DEATH BY NEGLIGENCE', 'OTHER IPC CRIMES', 'TOTAL IPC CRIMES'],
      dtype='object')

In [6]:
data['murder and attempt']=data.iloc[:, -30:-27].sum(axis=1)

In [7]:
data['Rape']=data.iloc[:, -28:-25].sum(axis=1)

In [8]:
data.drop(data.columns[[3,4,5,6,7,8]], axis = 1, inplace = True)

In [9]:
data['Kidnapping and Abduction']=data.iloc[:, -26:-23].sum(axis=1)

In [10]:
data.drop(data.columns[[3,4,5]], axis = 1, inplace = True)

In [11]:
data.drop(data.columns[[4,10,11,12,13,14]], axis = 1, inplace = True)

In [12]:
data['Dacoity and various theft']=data.iloc[:, -18:-12].sum(axis=1)

In [13]:
data.drop(data.columns[[3,4,5,6,7,8]], axis = 1, inplace = True)

In [14]:
data.drop(data.columns[[3,7,8,10,11]], axis = 1, inplace = True)

In [15]:
data['violence against women']=data.iloc[:, -8:-5].sum(axis=1)

In [16]:
data.drop(data.columns[[3,4,5]], axis = 1, inplace = True)

In [17]:
data.drop(data.columns[[3]], axis=1, inplace = True)

In [18]:
data.head()

,STATE/UT,DISTRICT,YEAR,murder and attempt,Rape,Kidnapping and Abduction,Dacoity and various theft,violence against women
0,ANDHRA PRADESH,ADILABAD,2012,175,176,164,1002,252
1,ANDHRA PRADESH,ANANTAPUR,2012,299,36,148,1393,222
2,ANDHRA PRADESH,CHITTOOR,2012,171,56,112,1151,392
3,ANDHRA PRADESH,CUDDAPAH,2012,216,52,152,1175,267
4,ANDHRA PRADESH,CYBERABAD,2012,321,150,316,10123,510


In [19]:
data.rename(columns = {'STATE/UT':'STATE'}, inplace = True)
data.head()

,STATE,DISTRICT,YEAR,murder and attempt,Rape,Kidnapping and Abduction,Dacoity and various theft,violence against women
0,ANDHRA PRADESH,ADILABAD,2012,175,176,164,1002,252
1,ANDHRA PRADESH,ANANTAPUR,2012,299,36,148,1393,222
2,ANDHRA PRADESH,CHITTOOR,2012,171,56,112,1151,392
3,ANDHRA PRADESH,CUDDAPAH,2012,216,52,152,1175,267
4,ANDHRA PRADESH,CYBERABAD,2012,321,150,316,10123,510


In [20]:
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()

data['STATE']= label_encoder.fit_transform(data['STATE'])
data['DISTRICT']= label_encoder.fit_transform(data['DISTRICT']) 
print(data.head())

   STATE  DISTRICT  YEAR  murder and attempt  Rape  Kidnapping and Abduction  \
0      2         2  2012                 175   176                       164   
1      2        28  2012                 299    36                       148   
2      2       147  2012                 171    56                       112   
3      2       164  2012                 216    52                       152   
4      2       166  2012                 321   150                       316   

   Dacoity and various theft  violence against women  
0                       1002                     252  
1                       1393                     222  
2                       1151                     392  
3                       1175                     267  
4                      10123                     510  


In [21]:
X = data.iloc[:,[0,1,2]].values
X

array([[   2,    2, 2012],
       [   2,   28, 2012],
       [   2,  147, 2012],
       ...,
       [  54,  369, 2013],
       [  54,  566, 2013],
       [  54,  780, 2013]], dtype=int64)

In [22]:
Y = data.iloc[:,[3,4,5,6,7]].values

In [23]:
Y

array([[ 175,  176,  164, 1002,  252],
       [ 299,   36,  148, 1393,  222],
       [ 171,   56,  112, 1151,  392],
       ...,
       [   9,   12,    6,  133,    3],
       [  35,   22,   76, 1137,   18],
       [  44,   34,   82, 1270,   21]], dtype=int64)

In [24]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=50)

In [25]:
rfc = RandomForestClassifier(n_estimators=100, class_weight="balanced")
multi_target_forest = MultiOutputClassifier(rfc, n_jobs=-1)

In [26]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1634 entries, 0 to 1633
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype
---  ------                     --------------  -----
 0   STATE                      1634 non-null   int32
 1   DISTRICT                   1634 non-null   int32
 2   YEAR                       1634 non-null   int64
 3   murder and attempt         1634 non-null   int64
 4   Rape                       1634 non-null   int64
 5   Kidnapping and Abduction   1634 non-null   int64
 6   Dacoity and various theft  1634 non-null   int64
 7   violence against women     1634 non-null   int64
dtypes: int32(2), int64(6)
memory usage: 89.5 KB


In [27]:
X.shape

(1634, 3)

In [28]:
X_train.shape

(1307, 3)

In [29]:
multi_target_forest.fit(X_train, Y_train)

MultiOutputClassifier(estimator=RandomForestClassifier(class_weight='balanced'),
                      n_jobs=-1)

In [30]:
y = multi_target_forest.predict(X_test)
acc_list = []
for i in range(Y_test.shape[0]):
        set_true = set( np.where(Y_train[i])[0] )
        set_pred = set( np.where(y[i])[0] )
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        acc_list.append(tmp_a)

In [31]:
print('Random forests score:', np.mean(acc_list)*100)

Random forests score: 92.04892966360856


In [32]:
y

array([[ 213,  134,  332, 1187,  131],
       [  89,  136,    8,  611,  162],
       [ 181,   46,  180, 2005,  361],
       ...,
       [  92,   12,   34,   25,    7],
       [   4,    0,   10,   44,    0],
       [ 203,  106,  244,  502,   36]], dtype=int64)

In [33]:
Y_test

array([[ 119,   66,   62,  671,   19],
       [   1,    0,    8,  392,   58],
       [ 180,  104,  178, 1956,  530],
       ...,
       [  75,   90,  522, 3369,  170],
       [  13,   12,   24,   90,    6],
       [  97,   10,   52,  156,   13]], dtype=int64)

In [34]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)

In [35]:
multi_target_tree = MultiOutputClassifier(clf, n_jobs=-1)
multi_target_tree.fit(X_train, Y_train)

MultiOutputClassifier(estimator=DecisionTreeClassifier(random_state=0),
                      n_jobs=-1)

In [36]:
y1 = multi_target_tree.predict(X_test)

In [37]:
acc_list = []
for i in range(Y_test.shape[0]):
        set_true = set( np.where(Y_train[i])[0] )
        set_pred = set( np.where(y1[i])[0] )
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        acc_list.append(tmp_a)

In [38]:
print('Decision tree score:', np.mean(acc_list)*100)

Decision tree score: 92.35474006116208


In [39]:
knn= KNeighborsClassifier()
mtk= MultiOutputClassifier(knn,n_jobs=-1)
mtk.fit(X_train,Y_train)
y2= mtk.predict(X_test)

In [40]:
acc_list = []
for i in range(Y_test.shape[0]):
        set_true = set( np.where(Y_train[i])[0] )
        set_pred = set( np.where(y2[i])[0] )
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        acc_list.append(tmp_a)

In [41]:
print('KNN score:', np.mean(acc_list)*100)

KNN score: 84.35779816513762
